In [454]:
# Check version runing on Jupyter notebook
from platform import python_version
print(python_version())

3.7.9


### compute hash: https://www.fileformat.info/tool/hash.htm
### md4 description: http://practicalcryptography.com/hashes/md4-hash/

In [153]:
import struct

In [400]:
ranges = []
for i in range(16):
    ranges.append((512-(32+(i-1)*32+1),512-(32+i*32)))

In [401]:
for r in ranges:
    print(r)

(511, 480)
(479, 448)
(447, 416)
(415, 384)
(383, 352)
(351, 320)
(319, 288)
(287, 256)
(255, 224)
(223, 192)
(191, 160)
(159, 128)
(127, 96)
(95, 64)
(63, 32)
(31, 0)


In [408]:
print(ranges[0])
print(ranges[4])
print(ranges[8])
print(ranges[12])
print()
print(ranges[1])
print(ranges[5])
print(ranges[9])
print(ranges[13])
print()
print(ranges[2])
print(ranges[6])
print(ranges[10])
print(ranges[14])
print()
print(ranges[3])
print(ranges[7])
print(ranges[11])
print(ranges[15])

(511, 480)
(383, 352)
(255, 224)
(127, 96)

(479, 448)
(351, 320)
(223, 192)
(95, 64)

(447, 416)
(319, 288)
(191, 160)
(63, 32)

(415, 384)
(287, 256)
(159, 128)
(31, 0)


In [407]:
print(ranges[0])
print(ranges[8])
print(ranges[4])
print(ranges[12])
print()
print(ranges[2])
print(ranges[10])
print(ranges[6])
print(ranges[14])
print()
print(ranges[1])
print(ranges[9])
print(ranges[5])
print(ranges[13])
print()
print(ranges[3])
print(ranges[11])
print(ranges[7])
print(ranges[15])

(511, 480)
(255, 224)
(383, 352)
(127, 96)

(447, 416)
(191, 160)
(319, 288)
(63, 32)

(479, 448)
(223, 192)
(351, 320)
(95, 64)

(415, 384)
(159, 128)
(287, 256)
(31, 0)


In [426]:
class MD4:
    """An implementation of the MD4 hash algorithm."""

    width = 32
    mask = 0xFFFFFFFF

    # Unlike, say, SHA-1, MD4 uses little-endian. Fascinating!
    h = [0x67452301, 0xEFCDAB89, 0x98BADCFE, 0x10325476]

    def __init__(self, msg=None):
        """:param ByteString msg: The message to be hashed."""
        if msg is None:
            msg = b""

        self.msg = msg

        # Pre-processing: Total length is a multiple of 512 bits.
        ml = len(msg) * 8
        msg += b"\x80"
        msg += b"\x00" * (-(len(msg) + 8) % 64)
        msg += struct.pack("<Q", ml)

        print("msg: ","".join(f"{value:02x}" for value in msg))
        print("msg: "," ".join(f"{value:02x}" for value in msg))
        print()
        

        # Process the message in successive 512-bit chunks.
        chunks = [msg[i : i + 64] for i in range(0, len(msg), 64)]        
        self._process(chunks)

    def __repr__(self):
        if self.msg:
            return f"{self.__class__.__name__}({self.msg:s})"
        return f"{self.__class__.__name__}()"

    def __str__(self):
        return self.hexdigest()

    def __eq__(self, other):
        return self.h == other.h

    def bytes(self):
        """:return: The final hash value as a `bytes` object."""
        ret = struct.pack("<4L", *self.h)
        print("final h: "," ".join(f"{value:02x}" for value in self.h))
        print("ret: "," ".join(f"{value:02x}" for value in ret))
        return ret

    def hexbytes(self):
        """:return: The final hash value as hexbytes."""
        return self.hexdigest().encode

    def hexdigest(self):
        """:return: The final hash value as a hexstring."""
        b = self.bytes()

        return "".join(f"{value:02x}" for value in b)

    def _process(self, chunks):
        for chunk in chunks:
            X, h = list(struct.unpack("<16I", chunk)), self.h.copy()
            
            print("initial h: ","".join(f"{value:02x}" for value in h))
            print()
            print("X"," ".join(f"{value:2x}" for value in X))
            print()

            # Round 1.
            Xi = [3, 7, 11, 19]
            for n in range(16):
                i, j, k, l = map(lambda x: x % 4, range(-n, -n + 4))
                K, S = n, Xi[n % 4]
                print("a",f"{h[i]:02x}")
                print("b",f"{h[j]:02x}")
                print("c",f"{h[k]:02x}")
                print("d",f"{h[l]:02x}")
                print("x",f"{X[K]:02x}")
                print("s",S)
                hn = h[i] + MD4.F(h[j], h[k], h[l]) + X[K]
                buf = MD4.lrot(hn & MD4.mask, S)
                h[i] = buf
                print("res",f"{buf:02x}")
                print("round 1, n",n,"h: ","".join(f"{value:02x}" for value in h))
                print()

            # Round 2.
            Xi = [3, 5, 9, 13]
            for n in range(16):
                i, j, k, l = map(lambda x: x % 4, range(-n, -n + 4))
                K, S = n % 4 * 4 + n // 4, Xi[n % 4]
                
                hn = h[i] + MD4.G(h[j], h[k], h[l]) + X[K] + 0x5A827999
                buf = MD4.lrot(hn & MD4.mask, S)
                h[i] = buf
                print("res",f"{buf:02x}")
                print("round 2, n",n,"h: ","".join(f"{value:02x}" for value in h))
                print()

            # Round 3.
            Xi = [3, 9, 11, 15]
            Ki = [0, 8, 4, 12, 2, 10, 6, 14, 1, 9, 5, 13, 3, 11, 7, 15]
            for n in range(16):
                i, j, k, l = map(lambda x: x % 4, range(-n, -n + 4))
                K, S = Ki[n], Xi[n % 4]
                
                hn = h[i] + MD4.H(h[j], h[k], h[l]) + X[K] + 0x6ED9EBA1
                buf = MD4.lrot(hn & MD4.mask, S)
                h[i] = buf
                print("res",f"{buf:02x}")
                print("round 3, n",n,"h: ","".join(f"{value:02x}" for value in h))
                print()

            self.h = [((v + n) & MD4.mask) for v, n in zip(self.h, h)]
            
            
            print("final h: ","".join(f"{value:02x}" for value in self.h))

    @staticmethod
    def F(x, y, z):
        return (x & y) | (~x & z)

    @staticmethod
    def G(x, y, z):
        return (x & y) | (x & z) | (y & z)

    @staticmethod
    def H(x, y, z):
        return x ^ y ^ z

    @staticmethod
    def lrot(value, n):
        lbits, rbits = (value << n) & MD4.mask, value >> (MD4.width - n)
        return lbits | rbits


In [453]:
message = "a"
MD4(message.encode()).hexdigest()

msg:  61800000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000800000000000000
msg:  61 80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 08 00 00 00 00 00 00 00

initial h:  67452301efcdab8998badcfe10325476

X 8061  0  0  0  0  0  0  0  0  0  0  0  0  0  8  0

a 67452301
b efcdab89
c 98badcfe
d 10325476
x 8061
s 3
res 40300
round 1, n 0 h:  40300efcdab8998badcfe10325476

a 10325476
b 40300
c efcdab89
d 98badcfe
x 00
s 7
res 789a3a54
round 1, n 1 h:  40300efcdab8998badcfe789a3a54

a 98badcfe
b 789a3a54
c 40300
d efcdab89
x 00
s 11
res 3043900
round 1, n 2 h:  40300efcdab893043900789a3a54

a efcdab89
b 3043900
c 789a3a54
d 40300
x 00
s 19
res 2c4f7e6f
round 1, n 3 h:  403002c4f7e6f3043900789a3a54

a 40300
b 2c4f7e6f
c 3043900
d 789a3a54
x 00
s 3
res 84c1d882
round 1, n 4 h:  84c1d8822c4f7e6f3043900789a3a54

a 789a3a54


'bde52cb31de33e46245e05fbdbd6fb24'

In [152]:
msg = "a"
msg = msg.encode()
ml = len(msg) * 8
msg += b"\x80"
msg += b"\x00" * (-(len(msg) + 8) % 64)
msg += struct.pack("<Q", ml)
print("h"+"".join(f"{value:02x}" for value in msg))

h61800000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000800000000000000


In [76]:
import struct
from enum import Enum
from math import (
    floor,
    sin,
)

from bitarray import bitarray


class MD5Buffer(Enum):
    A = 0x67452301
    B = 0xEFCDAB89
    C = 0x98BADCFE
    D = 0x10325476


class MD5(object):
    _string = None
    _buffers = {
        MD5Buffer.A: None,
        MD5Buffer.B: None,
        MD5Buffer.C: None,
        MD5Buffer.D: None,
    }

    @classmethod
    def hash(cls, string):
        cls._string = string

        preprocessed_bit_array = cls._step_2(cls._step_1())
        cls._step_3()
        cls._step_4(preprocessed_bit_array)
        return cls._step_5()

    @classmethod
    def _step_1(cls):
        # Convert the string to a bit array.
        bit_array = bitarray(endian="big")
        bit_array.frombytes(cls._string.encode("utf-8"))

        # Pad the string with a 1 bit and as many 0 bits required such that
        # the length of the bit array becomes congruent to 448 modulo 512.
        # Note that padding is always performed, even if the string's bit
        # length is already conguent to 448 modulo 512, which leads to a
        # new 512-bit message block.
        bit_array.append(1)
        while len(bit_array) % 512 != 448:
            bit_array.append(0)

        # For the remainder of the MD5 algorithm, all values are in
        # little endian, so transform the bit array to little endian.
        return bitarray(bit_array, endian="little")

    @classmethod
    def _step_2(cls, step_1_result):
        # Extend the result from step 1 with a 64-bit little endian
        # representation of the original message length (modulo 2^64).
        length = (len(cls._string) * 8) % pow(2, 64)
        length_bit_array = bitarray(endian="little")
        length_bit_array.frombytes(struct.pack("<Q", length))

        result = step_1_result.copy()
        result.extend(length_bit_array)
        return result

    @classmethod
    def _step_3(cls):
        # Initialize the buffers to their default values.
        for buffer_type in cls._buffers.keys():
            cls._buffers[buffer_type] = buffer_type.value

    @classmethod
    def _step_4(cls, step_2_result):
        # Define the four auxiliary functions that produce one 32-bit word.
        F = lambda x, y, z: (x & y) | (~x & z)
        G = lambda x, y, z: (x & z) | (y & ~z)
        H = lambda x, y, z: x ^ y ^ z
        I = lambda x, y, z: y ^ (x | ~z)

        # Define the left rotation function, which rotates `x` left `n` bits.
        rotate_left = lambda x, n: (x << n) | (x >> (32 - n))

        # Define a function for modular addition.
        modular_add = lambda a, b: (a + b) % pow(2, 32)

        # Compute the T table from the sine function. Note that the
        # RFC starts at index 1, but we start at index 0.
        T = [floor(pow(2, 32) * abs(sin(i + 1))) for i in range(64)]

        # The total number of 32-bit words to process, N, is always a
        # multiple of 16.
        N = len(step_2_result) // 32

        # Process chunks of 512 bits.
        for chunk_index in range(N // 16):
            # Break the chunk into 16 words of 32 bits in list X.
            start = chunk_index * 512
            X = [step_2_result[start + (x * 32) : start + (x * 32) + 32] for x in range(16)]

            # Convert the `bitarray` objects to integers.
            X = [int.from_bytes(word.tobytes(), byteorder="little") for word in X]

            # Make shorthands for the buffers A, B, C and D.
            A = cls._buffers[MD5Buffer.A]
            B = cls._buffers[MD5Buffer.B]
            C = cls._buffers[MD5Buffer.C]
            D = cls._buffers[MD5Buffer.D]

            # Execute the four rounds with 16 operations each.
            for i in range(4 * 16):
                if 0 <= i <= 15:
                    k = i
                    s = [7, 12, 17, 22]
                    temp = F(B, C, D)
                elif 16 <= i <= 31:
                    k = ((5 * i) + 1) % 16
                    s = [5, 9, 14, 20]
                    temp = G(B, C, D)
                elif 32 <= i <= 47:
                    k = ((3 * i) + 5) % 16
                    s = [4, 11, 16, 23]
                    temp = H(B, C, D)
                elif 48 <= i <= 63:
                    k = (7 * i) % 16
                    s = [6, 10, 15, 21]
                    temp = I(B, C, D)

                # The MD5 algorithm uses modular addition. Note that we need a
                # temporary variable here. If we would put the result in `A`, then
                # the expression `A = D` below would overwrite it. We also cannot
                # move `A = D` lower because the original `D` would already have
                # been overwritten by the `D = C` expression.
                temp = modular_add(temp, X[k])
                temp = modular_add(temp, T[i])
                temp = modular_add(temp, A)
                temp = rotate_left(temp, s[i % 4])
                temp = modular_add(temp, B)

                # Swap the registers for the next operation.
                A = D
                D = C
                C = B
                B = temp

            # Update the buffers with the results from this chunk.
            cls._buffers[MD5Buffer.A] = modular_add(cls._buffers[MD5Buffer.A], A)
            cls._buffers[MD5Buffer.B] = modular_add(cls._buffers[MD5Buffer.B], B)
            cls._buffers[MD5Buffer.C] = modular_add(cls._buffers[MD5Buffer.C], C)
            cls._buffers[MD5Buffer.D] = modular_add(cls._buffers[MD5Buffer.D], D)

    @classmethod
    def _step_5(cls):
        # Convert the buffers to little-endian.
        A = struct.unpack("<I", struct.pack(">I", cls._buffers[MD5Buffer.A]))[0]
        B = struct.unpack("<I", struct.pack(">I", cls._buffers[MD5Buffer.B]))[0]
        C = struct.unpack("<I", struct.pack(">I", cls._buffers[MD5Buffer.C]))[0]
        D = struct.unpack("<I", struct.pack(">I", cls._buffers[MD5Buffer.D]))[0]

        # Output the buffers in lower-case hexadecimal format.
        return f"{format(A, '08x')}{format(B, '08x')}{format(C, '08x')}{format(D, '08x')}"


In [77]:
msg = "a" 
MD5.hash(msg)

'0cc175b9c0f1b6a831c399e269772661'